# This notebook can be used to calculate the consensus differences on each chromosomal arm between two sets of Anopheles gambiae. The example data are the potential fathers of the crosses described in table 3 of Dyer et al 2023

In [ ]:
#it is advisable to run this notebook using juputer-notebook not google colab as it usually runs out 
#of resources when filtering positions and crashes on colab

# Setup access

In [1]:
# install packages - will take a few minutes to compile scikit-allel
!pip install -U zarr==2.6.1 fsspec==0.8.7 gcsfs==0.7.2 dask==2021.03.0 plotly scikit-allel malariagen_data

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.7/131.7 KB 6.3 MB/s eta 0:00:00
  Using cached dask-2023.5.0-py3-none-any.whl (1.2 MB)


  Using cached dask-2023.4.1-py3-none-any.whl (1.2 MB)
  Using cached dask-2023.4.0-py3-none-any.whl (1.2 MB)
  Using cached dask-2023.3.2-py3-none-any.whl (1.2 MB)
  Using cached dask-2023.3.1-py3-none-any.whl (1.2 MB)
  Using cached dask-2023.3.0-py3-none-any.whl (1.2 MB)
  Using cached dask-2023.2.1-py3-none-any.whl (1.2 MB)
  Using cached dask-2023.2.0-py3-none-any.whl (1.2 MB)
  Using cached dask-2023.1.1-py3-none-any.whl (1.1 MB)
  Using cached dask-2023.1.0-py3-none-any.whl (1.1 MB)
  Using cached dask-2022.12.1-py3-none-any.whl (1.1 MB)
  Using cached dask-2022.12.0-py3-none-any.whl (1.1 MB)
  Using cached dask-2022.11.1-py3-none-any.whl (1.1 MB)
  Using cached dask-2022.11.0-py3-none-any.whl (1.1 MB)
  Using cached dask-2022.10.2-py3-none-any.whl (1.1 MB)
  Using cached dask-2022.10.0-py3-none-any.whl (1.1 MB)
  Using cached dask-2022.9.2-py3-none-any.whl (1.1 MB)
  Using cached dask-2022.9.1-py3-none-any.whl (1.1 MB)
  Using cached dask-2022.9.0-py3-none-any.whl (1.1 MB)
  Us

  Attempting uninstall: malariagen_data
    Found existing installation: malariagen-data 7.10.0
    Uninstalling malariagen-data-7.10.0:
      Successfully uninstalled malariagen-data-7.10.0


In [2]:
# import packages
import os
import bisect
import hashlib
import json
import allel
import numpy as np
import dask
import dask.array as da
from dask.diagnostics import ProgressBar
# quieten dask warnings about large chunks
dask.config.set(**{'array.slicing.split_large_chunks': False})
import pandas as pd
import malariagen_data
import gcsfs
import zarr

Not all packages are needed yet but they might come in handy later on

In [3]:
input_path = 'vo_agam_release/v3/snp_genotypes/per_sample/AG1000G-X/zarr/potential_fathers_combined.zarr'

In [4]:
# Passing session credentials via token prevents KilledWorker errors due to authentication failure.
# In case you decide to use Dask workers
gcs_session = gcsfs.GCSFileSystem(
    project='malariagen-jupyterhub',
    token='cache',
    cache_timeout=0,
    block_size=2**18,  # minimum, reduces memory footprint when reading from zip files
)
gcs = gcsfs.GCSFileSystem(
    project='malariagen-jupyterhub',
    token=gcs_session.credentials,
    cache_timeout=0,
    block_size=2**18,  # minimum, reduces memory footprint when reading from zip files
)

In [5]:
# Eyeball a few objects in the vo_agam_production bucket to check the GCS connection
gcs.ls('vo_agam_release')[:3]

['vo_agam_release/reference',
 'vo_agam_release/v3',
 'vo_agam_release/v3-config.json']

In [6]:
# Open the zarr store at output_path for ouput, using read-only mode and check that it is correct
gcs_input_store = gcs.get_mapper(input_path)
input_zarr_root = zarr.open_group(store=gcs_input_store, mode='r')
len(list(input_zarr_root['samples']))

26

In [7]:
#check the names of the samples
#output should be ['AC0394-C', 'AC0395-C', 'AC0396-C', 'AC0397-C', 'AC0398-C', 'AC0399-C', 'AC0400-C', 'AC0401-C', 'AC0402-C', 'AC0403-C', 'AC0404-C', 'AC0405-C', 'AC0406-C', 'AC0407-C', 'AC0408-C', 'AC0409-C', 'AC0410-C', 'AC0411-C', 'AC0412-C', 'AC0413-C', 'AC0414-C', 'AC0415-C', 'AC0416-C', 'AC0417-C', 'AC0418-C', 'AC0419-C']
print(list(input_zarr_root['samples']))

['AC0394-C', 'AC0395-C', 'AC0396-C', 'AC0397-C', 'AC0398-C', 'AC0399-C', 'AC0400-C', 'AC0401-C', 'AC0402-C', 'AC0403-C', 'AC0404-C', 'AC0405-C', 'AC0406-C', 'AC0407-C', 'AC0408-C', 'AC0409-C', 'AC0410-C', 'AC0411-C', 'AC0412-C', 'AC0413-C', 'AC0414-C', 'AC0415-C', 'AC0416-C', 'AC0417-C', 'AC0418-C', 'AC0419-C']


In [ ]:
#note individuals AC0394-C-AC0406-C are potential fathers from Nagongera colony which were crossed to Kisumu females
#and AC0407-C-AC0419-C are potential fathers from Kisumu colony which were crossed with Nagongera females

# Access the genotypes for the Kisumu and Nagongera potential fathers

In [8]:
# Access the genotypes 2L
k_array_2L = input_zarr_root['2L/calldata/GT'][:,:13,:]
b_array_2L = input_zarr_root['2L/calldata/GT'][:,13:,:]

In [9]:
# Access the genotype 2R
k_array_2R = input_zarr_root['2R/calldata/GT'][:,:13,:]
b_array_2R = input_zarr_root['2R/calldata/GT'][:,13:,:]

In [10]:
# Access the genotype 2R
k_array_3L = input_zarr_root['3L/calldata/GT'][:,:13,:]
b_array_3L = input_zarr_root['3L/calldata/GT'][:,13:,:]

In [11]:
#Access the genotype 3R
k_array_3R = input_zarr_root['3R/calldata/GT'][:,:13,:]
b_array_3R = input_zarr_root['3R/calldata/GT'][:,13:,:]

In [12]:
#Access the genotype X
k_array_X = input_zarr_root['X/calldata/GT'][:,:13,:]
b_array_X = input_zarr_root['X/calldata/GT'][:,13:,:]

# Convert genotypes to genotype dask arrays

In [13]:
k_gt_2L = allel.GenotypeDaskArray(k_array_2L)
k_gt_2L

<GenotypeDaskArray shape=(48525747, 13, 2) dtype=int8>

In [14]:
b_gt_2L = allel.GenotypeDaskArray(b_array_2L)
b_gt_2L

<GenotypeDaskArray shape=(48525747, 13, 2) dtype=int8>

In [15]:
k_gt_2R = allel.GenotypeDaskArray(k_array_2R)
k_gt_2R

<GenotypeDaskArray shape=(60132453, 13, 2) dtype=int8>

In [16]:
b_gt_2R = allel.GenotypeDaskArray(b_array_2R)
b_gt_2R

<GenotypeDaskArray shape=(60132453, 13, 2) dtype=int8>

In [17]:
k_gt_3L = allel.GenotypeDaskArray(k_array_3L)
k_gt_3L

<GenotypeDaskArray shape=(40758473, 13, 2) dtype=int8>

In [18]:
b_gt_3L = allel.GenotypeDaskArray(b_array_3L)
b_gt_3L

<GenotypeDaskArray shape=(40758473, 13, 2) dtype=int8>

In [19]:
k_gt_3R = allel.GenotypeDaskArray(k_array_3R)
k_gt_3R

<GenotypeDaskArray shape=(52226568, 13, 2) dtype=int8>

In [20]:
b_gt_3R = allel.GenotypeDaskArray(b_array_3R)
b_gt_3R

<GenotypeDaskArray shape=(52226568, 13, 2) dtype=int8>

In [21]:
k_gt_X = allel.GenotypeDaskArray(k_array_X)
k_gt_X

<GenotypeDaskArray shape=(23385349, 13, 2) dtype=int8>

In [22]:
b_gt_X = allel.GenotypeDaskArray(b_array_X)
b_gt_X

<GenotypeDaskArray shape=(23385349, 13, 2) dtype=int8>

# Set up loc pass filter to choose sites passing gamb_colu filter

In [24]:
ag3 = malariagen_data.Ag3("gs://vo_agam_release/")

In [25]:
ds_snps = ag3.snp_calls
#ds_snps

In [26]:
ds_snps2L = ag3.snp_calls(region="2L")
#ds_snps2L

In [27]:
ds_snps2R = ag3.snp_calls(region="2R")
ds_snps2R

<xarray.Dataset>
Dimensions:                             (variants: 60132453, alleles: 4,
                                         samples: 3081, ploidy: 2)
Coordinates:
    variant_position                    (variants) int32 dask.array<chunksize=(524288,), meta=np.ndarray>
    variant_contig                      (variants) uint8 dask.array<chunksize=(524288,), meta=np.ndarray>
    sample_id                           (samples) <U24 dask.array<chunksize=(81,), meta=np.ndarray>
Dimensions without coordinates: variants, alleles, samples, ploidy
Data variables:
    variant_allele                      (variants, alleles) |S1 dask.array<chunksize=(524288, 1), meta=np.ndarray>
    variant_filter_pass_gamb_colu_arab  (variants) bool dask.array<chunksize=(300000,), meta=np.ndarray>
    variant_filter_pass_gamb_colu       (variants) bool dask.array<chunksize=(300000,), meta=np.ndarray>
    variant_filter_pass_arab            (variants) bool dask.array<chunksize=(300000,), meta=np.ndarray>
    call_genotype                       (variants, samples, ploidy) int8 dask.array<chunksize=(300000, 50, 2), meta=np.ndarray>
    call_GQ                             (variants, samples) int16 dask.array<chunksize=(300000, 50), meta=np.ndarray>
    call_MQ                             (variants, samples) int16 dask.array<chunksize=(300000, 50), meta=np.ndarray>
    call_AD                             (variants, samples, alleles) int16 dask.array<chunksize=(300000, 50, 4), meta=np.ndarray>
    call_genotype_mask                  (variants, samples, ploidy) bool dask.array<chunksize=(300000, 50, 2), meta=np.ndarray>
Attributes:
    contigs:  ('2R', '2L', '3R', '3L', 'X')

In [28]:
ds_snps3L = ag3.snp_calls(region="3L")
#ds_snps3L

In [29]:
ds_snps3R = ag3.snp_calls(region="3R")
#ds_snps3R

In [30]:
ds_snpsX = ag3.snp_calls(region="X", sample_sets="3.0")
#ds_snpsX

In [31]:
#set up loc pass filter to choose sites passing gamb_colu filter
loc_pass2L = ds_snps2L["variant_filter_pass_gamb_colu"].data
loc_pass2L

dask.array<array, shape=(48525747,), dtype=bool, chunksize=(300000,), chunktype=numpy.ndarray>

In [32]:
#set up loc pass filter to choose sites passing gamb_colu filter
loc_pass2R = ds_snps2R["variant_filter_pass_gamb_colu"].data
loc_pass2R

dask.array<array, shape=(60132453,), dtype=bool, chunksize=(300000,), chunktype=numpy.ndarray>

In [33]:
#set up loc pass filter to choose sites passing gamb_colu filter
loc_pass3L = ds_snps3L["variant_filter_pass_gamb_colu"].data
loc_pass3L

dask.array<array, shape=(40758473,), dtype=bool, chunksize=(300000,), chunktype=numpy.ndarray>

In [34]:
#set up loc pass filter to choose sites passing gamb_colu filter
loc_pass3R = ds_snps3R["variant_filter_pass_gamb_colu"].data
loc_pass3R

dask.array<array, shape=(52226568,), dtype=bool, chunksize=(300000,), chunktype=numpy.ndarray>

In [35]:
#set up loc pass filter to choose sites passing gamb_colu filter
loc_passX = ds_snpsX["variant_filter_pass_gamb_colu"].data
loc_passX

dask.array<array, shape=(23385349,), dtype=bool, chunksize=(300000,), chunktype=numpy.ndarray>

# Filter genotype data for sites passing gamb-colu filter

In [36]:
#compress the Nagongera fathers array by the gamb colu loc pass filter
#google colab usually crashes due to insufficient resources at this point 
#best run in jupyter-notebook
k_array_compressed_2L=k_gt_2L.compress(loc_pass2L)
k_array_compressed_2L

<GenotypeDaskArray shape=(36005131, 13, 2) dtype=int8>

In [37]:
#compress the Nagongera fathers array by the gamb colu loc pass filter
b_array_compressed_2L=b_gt_2L.compress(loc_pass2L)
b_array_compressed_2L

<GenotypeDaskArray shape=(36005131, 13, 2) dtype=int8>

In [38]:
#compress the Nagongera fathers array by the gamb colu loc pass filter
k_array_compressed_2R=k_gt_2R.compress(loc_pass2R)
k_array_compressed_2R

<GenotypeDaskArray shape=(44439759, 13, 2) dtype=int8>

In [39]:
#compress the Nagongera fathers array by the gamb colu loc pass filter
b_array_compressed_2R=b_gt_2R.compress(loc_pass2R)
b_array_compressed_2R

<GenotypeDaskArray shape=(44439759, 13, 2) dtype=int8>

In [47]:
#compress the Nagongera fathers array by the gamb colu loc pass filter
k_array_compressed_3L=k_gt_3L.compress(loc_pass3L)
k_array_compressed_3L

<GenotypeDaskArray shape=(28707856, 13, 2) dtype=int8>

In [48]:
#compress the Nagongera fathers array by the gamb colu loc pass filter
b_array_compressed_3L=b_gt_3L.compress(loc_pass3L)
b_array_compressed_3L

<GenotypeDaskArray shape=(28707856, 13, 2) dtype=int8>

In [49]:
#compress the Nagongera fathers array by the gamb colu loc pass filter
k_array_compressed_3R=k_gt_3R.compress(loc_pass3R)
k_array_compressed_3R

<GenotypeDaskArray shape=(37199402, 13, 2) dtype=int8>

In [50]:
#compress the Nagongera fathers array by the gamb colu loc pass filter
b_array_compressed_3R=b_gt_3R.compress(loc_pass3R)
b_array_compressed_3R

<GenotypeDaskArray shape=(37199402, 13, 2) dtype=int8>

In [46]:
#compress the Nagongera fathers array by the gamb colu loc pass filter
k_array_compressed_X=k_gt_X.compress(loc_passX)
k_array_compressed_X

<GenotypeDaskArray shape=(16362809, 13, 2) dtype=int8>

In [51]:
#compress the Nagongera fathers array by the gamb colu loc pass filter
b_array_compressed_X=b_gt_X.compress(loc_passX)
b_array_compressed_X

<GenotypeDaskArray shape=(16362809, 13, 2) dtype=int8>

# alternative methods to find non seg and seg sites

In [52]:
# An allele is the consensus for a cross if all samples are homozygotes (with that allele)
def is_consensus_val(array, ref):
  t0 = array[:,:,0] == ref
  t1 = array[:,:,1] == ref
  t = da.logical_and(t0,t1)
  return np.sum(t, axis = 1) == array.shape[1]

In [53]:
# An allele is the consensus for a cross if most samples are homozygotes (with that allele)
def is_nearly_consensus_val(array, ref, tolerance):
  t0 = array[:,:,0] == ref
  t1 = array[:,:,1] == ref
  t = da.logical_and(t0,t1)
  return np.sum(t, axis = 1) > array.shape[1]-1-tolerance

In [56]:
k_array_compressed_2L

<GenotypeDaskArray shape=(36005131, 13, 2) dtype=int8>

In [57]:
# Compute consensus for K crosses
ick_missing = is_consensus_val(k_array_compressed_2L,-1)
ick0 = is_consensus_val(k_array_compressed_2L,0)
ick1 = is_consensus_val(k_array_compressed_2L,1)
ick2 = is_consensus_val(k_array_compressed_2L,2)
ick3 = is_consensus_val(k_array_compressed_2L,3)

In [81]:
k_arrays = [k_array_compressed_2L,
            k_array_compressed_2R,
            k_array_compressed_3L,
            k_array_compressed_3R,
           k_array_compressed_X]

In [84]:
k_arrays[0]

<GenotypeDaskArray shape=(36005131, 13, 2) dtype=int8>

In [85]:
is_consensus_val(k_arrays[0],0)

dask.array<eq, shape=(36005131,), dtype=bool, chunksize=(156111,), chunktype=numpy.ndarray>

In [129]:
def compute_K_consensus(k_array):
    ick_missing = is_consensus_val(k_array,-1)
    ick0 = is_consensus_val(k_array,0)
    ick1 = is_consensus_val(k_array,1)
    ick2 = is_consensus_val(k_array,2)
    ick3 = is_consensus_val(k_array,3)
    
    return ick0

In [143]:
ick3

dask.array<eq, shape=(28707856,), dtype=bool, chunksize=(153780,), chunktype=numpy.ndarray>

In [108]:
#this is not used later - the arrays should be of the same length. Missing data is coded as -1
def compute_B_consensus(b_array):
    ick_missing = is_consensus_val(b_array,-1)
    icb0 = is_consensus_val(b_array,0)
    icb1 = is_consensus_val(b_array,1)
    icb2 = is_consensus_val(b_array,2)
    icb3 = is_consensus_val(b_array,3)
    
    return icb_missing, icb0, icb1, icb2,icb3

In [146]:
ick0 = compute_K_consensus(k_array_compressed_3L)
ick0

dask.array<eq, shape=(28707856,), dtype=bool, chunksize=(153780,), chunktype=numpy.ndarray>

In [147]:
#np.count_nonzero(ick0) is the number of hom ref sites
homrefK=np.count_nonzero(ick0)
int(homrefK)

27251448

In [148]:
#check how many are hom alt (ick1, 2 and 3 summed as 3 alternative alleles)
hom_alt_k=np.count_nonzero(ick1)+np.count_nonzero(ick2)+np.count_nonzero(ick3)
int(hom_alt_k)

67499

In [116]:
icb_missing, icb0, icb1, icb2,icb3 = compute_B_consensus(b_array_compressed_3L)

In [119]:
# Computes the sites where there is a different consensus between crosses
is_diff_consensus = da.zeros(len(ick0))
for val_k in range(0,4):
  for val_b in range(0,4):
    if val_k != val_b:
      is_diff_consensus = da.logical_or(is_diff_consensus, da.logical_and(is_consensus_val(k_array_compressed_3L,val_k), is_consensus_val(b_array_compressed_3L, val_b)))

In [ ]:
# Count such consensus using np.sum(consensus.compute())
np.sum(is_diff_consensus.compute())

# Define function to calculate consensus difference

In [123]:
def calculate_consensus_difference(ick0, k_array, b_array):
    # Computes the sites where there is a different consensus between crosses
    is_diff_consensus = da.zeros(len(ick0))
    for val_k in range(0,4):
      for val_b in range(0,4):
        if val_k != val_b:
          is_diff_consensus = da.logical_or(is_diff_consensus, da.logical_and(is_consensus_val(k_array,val_k), is_consensus_val(b_array, val_b)))
    return is_diff_consensus    

In [136]:
#calculate_consensus difference on 2L (with site filters)
ick0 = compute_K_consensus(k_array_compressed_2L)
concensus_diff_2L = calculate_consensus_difference(ick0, k_array_compressed_2L, b_array_compressed_2L)
np.sum(concensus_diff_2L.compute())

2575

In [135]:
#calculate_consensus difference on 2R (with site filters)
ick0 = compute_K_consensus(k_array_compressed_2R)
concensus_diff_2R = calculate_consensus_difference(ick0, k_array_compressed_2R, b_array_compressed_2R)
np.sum(concensus_diff_2R.compute())

13507

In [133]:
#calculate_consensus difference on 3L (with site filters)
ick0 = compute_K_consensus(k_array_compressed_3L)
concensus_diff_3L = calculate_consensus_difference(ick0, k_array_compressed_3L, b_array_compressed_3L)
np.sum(concensus_diff_3L.compute())

6216

In [130]:
#calculate_consensus difference on 3R (with site filters)
ick0 = compute_K_consensus(k_array_compressed_3R)
concensus_diff_3R = calculate_consensus_difference(ick0, k_array_compressed_3R, b_array_compressed_3R)
np.sum(concensus_diff_3R.compute())

2949

In [137]:
#calculate_consensus difference on X (with site filters)
ick0 = compute_K_consensus(k_array_compressed_X)
concensus_diff_X = calculate_consensus_difference(ick0, k_array_compressed_X, b_array_compressed_X)
np.sum(concensus_diff_X.compute())

10552

# Calculate consensus differences without site filter

In [140]:
#calculate_consensus difference on 3R (without site filters)
ick0 = compute_K_consensus(k_array_2L)
concensus_diff_2L = calculate_consensus_difference(ick0, k_array_2L, b_array_2L)
np.sum(concensus_diff_2L.compute())

4518

In [139]:
#calculate_consensus difference on 3R (without site filters)
ick0 = compute_K_consensus(k_array_2R)
concensus_diff_2R = calculate_consensus_difference(ick0, k_array_2R, b_array_2R)
np.sum(concensus_diff_2R.compute())

19716

In [134]:
#calculate_consensus difference on 3R (without site filters)
ick0 = compute_K_consensus(k_array_3L)
concensus_diff_3L = calculate_consensus_difference(ick0, k_array_3L, b_array_3L)
np.sum(concensus_diff_3L.compute())

8673

In [131]:
#calculate_consensus difference on 3R (without site filters)
ick0 = compute_K_consensus(k_array_3R)
concensus_diff_3R = calculate_consensus_difference(ick0, k_array_3R, b_array_3R)
np.sum(concensus_diff_3R.compute())

4614

In [138]:
#calculate_consensus difference on X (without site filters)
ick0 = compute_K_consensus(k_array_X)
concensus_diff_X = calculate_consensus_difference(ick0, k_array_X, b_array_X)
np.sum(concensus_diff_X.compute())

13889